In [1]:
import pandas as pd
import os
import shutil
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [2]:
def convertir_ugm3_a_ppb(ugm3, peso_molecular):
    # 24.45 es el volumen molar del gas ideal a 25°C y 1 atm en L/mol
    volumen_molar = 24.45
    ppb = (ugm3 * volumen_molar) / peso_molecular
    return ppb

### Selección de datos para analisis de tráfico

In [37]:
dir_raw_data = "C:/Users/valer/Documents/CIC/doctorado/air_pollution/preprocessing_airpollution_data/all_data_2005xstation_31_05_2024"

In [38]:
traffic_dir = "TRAFFIC_data"
if not os.path.exists(traffic_dir):
    os.makedirs(traffic_dir)
files = os.listdir(dir_raw_data)
for filename in files:
    file_path = os.path.join(dir_raw_data ,filename)
    df = pd.read_csv(file_path)
    #df = pd.read_csv(file_path, dtype={'date': 'str'})
    #df = pd.read_csv(file_path, parse_dates=['date'], date_parser=lambda x: pd.to_datetime(x, format='%Y/%m/%d %H:%M'))

    df['date'] = pd.to_datetime(df['date'], format='%Y/%m/%d %H:%M')
    #df['date'] = df['date'].dt.strftime('%Y/%m/%d %H:%M:%S')
    fecha_inicio = pd.Timestamp('2024-02-23')
    df = df[df['date'] > fecha_inicio].copy()
    new_name = os.path.join(traffic_dir, filename)
    print(filename,len(df))
    df.to_csv(new_name, index=False)

ACO.csv 0
AJM.csv 2359
AJU.csv 2214
ARA.csv 0
ATI.csv 2316
AZC.csv 0
BJU.csv 2302
CAM.csv 2101
CCA.csv 2300
CES.csv 0
CHO.csv 1876
COY.csv 0
CUA.csv 2317
CUT.csv 2344
FAC.csv 2318
FAR.csv 2345
GAM.csv 2355
HGM.csv 1773
IMP.csv 0
INN.csv 2323
IZT.csv 2323
LAG.csv 0
LLA.csv 2286
LPR.csv 2253
MER.csv 2231
MGH.csv 2359
MON.csv 2263
MPA.csv 2333
NEZ.csv 2356
PED.csv 2201
PLA.csv 0
SAC.csv 2364
SAG.csv 2226
SFE.csv 0
SJA.csv 0
SUR.csv 0
TAC.csv 0
TAH.csv 2327
TAX.csv 0
TEC.csv 0
TLA.csv 2140
TLI.csv 2345
TPN.csv 0
UAX.csv 1919
UIZ.csv 2307
VAL.csv 0
VIF.csv 2256
XAL.csv 1313


In [39]:
stations = ['AJM', 'CUA', 'CUT', 'FAC', 'MER', 'NEZ', 'PED', 'SAC', 'UIZ', 'VIF']
dir_traffic_preprocessed_data = "TRAFFIC_data/dir_traffic_preprocessed_data"
dir = "TRAFFIC_data"
if not os.path.exists(dir_traffic_preprocessed_data):
    os.makedirs(dir_traffic_preprocessed_data)
files = os.listdir(dir)
for file in files:
    if file.endswith(".csv"):
        print(file)

ACO.csv
AJM.csv
AJU.csv
ARA.csv
ATI.csv
AZC.csv
BJU.csv
CAM.csv
CCA.csv
CES.csv
CHO.csv
COY.csv
CUA.csv
CUT.csv
FAC.csv
FAR.csv
GAM.csv
HGM.csv
IMP.csv
INN.csv
IZT.csv
LAG.csv
LLA.csv
LPR.csv
MER.csv
MGH.csv
MON.csv
MPA.csv
NEZ.csv
PED.csv
PLA.csv
SAC.csv
SAG.csv
SFE.csv
SJA.csv
SUR.csv
TAC.csv
TAH.csv
TAX.csv
TEC.csv
TLA.csv
TLI.csv
TPN.csv
UAX.csv
UIZ.csv
VAL.csv
VIF.csv
XAL.csv


In [40]:
stations = ['AJM', 'CUA', 'CUT', 'FAC', 'MER', 'NEZ', 'PED', 'SAC', 'UIZ', 'VIF']

dir_traffic_preprocessed_data = "TRAFFIC_data/dir_traffic_preprocessed_data"
dir = "TRAFFIC_data"
if not os.path.exists(dir_traffic_preprocessed_data):
    os.makedirs(dir_traffic_preprocessed_data)

dir = 'TRAFFIC_data'  
files = os.listdir(dir)

info_datasets = []
columnas = ["file_name", "len_dataset","%datosFaltantes"]

for file_name in files:
    if file_name.endswith(".csv"):
        file_path = os.path.join(dir,file_name)
        print(file_path)
        dataset = pd.read_csv(file_path)
        if len(dataset) > 0:
            dataset[['year', 'month', 'day']] = dataset['date'].str.split('-', expand=True)
            dataset["hour"] = dataset["day"]
            dataset[["day", "hour"]] = dataset["day"].str.split(' ', expand=True)
            dataset[["hour","minute", "second"]] = dataset["hour"].str.split(':', expand=True)

            porcentaje_total_vacios = round(dataset.isna().mean().mean() * 100,2)
            info_datasets.append([file_name, len(dataset), porcentaje_total_vacios])
            
            #dataset = dataset.dropna(axis=1, how='all')
            df = dataset.copy()

            # Borra por columnas, si toda la columna esta vacía (suma 0) entonces la borra completa
            for i in dataset:
                suma = dataset[i].sum()
                if suma == 0:
                    df.drop(columns=[i], inplace=True)

            dataset = df
            # Borra por fila, Drop rows which contain missing values.
            dataset = dataset.dropna(axis=0)
            df = dataset[["date", "year", "month", "day", "hour", "minute"]].copy()
            dataset = dataset.drop(["date", "year", "month", "day", "hour", "minute"], axis=1)
            # Elimina aquellas columnas que tienen solo un valor único en todas sus filas.
            for column in dataset.columns:
                if dataset[column].nunique() == 1:
                    dataset = dataset.drop(columns=[column], axis=1)

            # Eliminación de las filas con valores negativos
            dataset = dataset[(dataset >= 0).all(axis=1)]

            # Cambio de unidades. Conversión de ppm a ppb
            if "CO" in dataset.columns:
                dataset["CO"] = dataset["CO"] * 1000

            if len(dataset) > 0 and len(dataset.columns) > 0:
                # Normalización
                scaler = MinMaxScaler()
                df_normalized = scaler.fit_transform(dataset)
                dataset = pd.DataFrame(df_normalized, columns=dataset.columns)
                df.reset_index(inplace=True, drop=True)

                dataset.insert(0, "date", df["date"])
                df = df.drop(["date"], axis=1)
                dataset[["year", "month", "day", "hour", "minute"]] = df    

                name = os.path.join(dir_traffic_preprocessed_data, file_name)
                dataset.to_csv(name, index=False)

info_name = "info_split_datasets_soloTrafico.csv"
info_datasets = pd.DataFrame(info_datasets, columns=columnas)
info_datasets.to_csv(info_name, index=False)

TRAFFIC_data\ACO.csv
TRAFFIC_data\AJM.csv
TRAFFIC_data\AJU.csv
TRAFFIC_data\ARA.csv
TRAFFIC_data\ATI.csv
TRAFFIC_data\AZC.csv
TRAFFIC_data\BJU.csv
TRAFFIC_data\CAM.csv
TRAFFIC_data\CCA.csv
TRAFFIC_data\CES.csv
TRAFFIC_data\CHO.csv
TRAFFIC_data\COY.csv
TRAFFIC_data\CUA.csv
TRAFFIC_data\CUT.csv
TRAFFIC_data\FAC.csv
TRAFFIC_data\FAR.csv
TRAFFIC_data\GAM.csv
TRAFFIC_data\HGM.csv
TRAFFIC_data\IMP.csv
TRAFFIC_data\INN.csv
TRAFFIC_data\IZT.csv
TRAFFIC_data\LAG.csv
TRAFFIC_data\LLA.csv
TRAFFIC_data\LPR.csv
TRAFFIC_data\MER.csv
TRAFFIC_data\MGH.csv
TRAFFIC_data\MON.csv
TRAFFIC_data\MPA.csv
TRAFFIC_data\NEZ.csv
TRAFFIC_data\PED.csv
TRAFFIC_data\PLA.csv
TRAFFIC_data\SAC.csv
TRAFFIC_data\SAG.csv
TRAFFIC_data\SFE.csv
TRAFFIC_data\SJA.csv
TRAFFIC_data\SUR.csv
TRAFFIC_data\TAC.csv
TRAFFIC_data\TAH.csv
TRAFFIC_data\TAX.csv
TRAFFIC_data\TEC.csv
TRAFFIC_data\TLA.csv
TRAFFIC_data\TLI.csv
TRAFFIC_data\TPN.csv
TRAFFIC_data\UAX.csv
TRAFFIC_data\UIZ.csv
TRAFFIC_data\VAL.csv
TRAFFIC_data\VIF.csv
TRAFFIC_data\

In [41]:
columnas_especificas = ['CO', 'NO', 'NOX', 'NO2', 'O3', 'PM10', 'PM25', 'RH', 'SO2', 'TMP', 'WDR', 'WSP']
dir = "TRAFFIC_data/dir_traffic_preprocessed_data"
files = os.listdir(dir)
lista = []
for file_name in files:
    columnas_cont = []
    print(file_name[:-4])
    file_path = os.path.join(dir,file_name)
    df = pd.read_csv(file_path)
    valores_minimos = df.min()
    valores_maximos = df.max()
    df.drop(columns=["date", "year", "month", "day", "hour", "minute"], inplace=True)
    print(valores_minimos.date, valores_maximos.date, len(df), df.columns)
    columnas_cont = [file_name[:-4], valores_minimos.date, valores_minimos.year, valores_maximos.date, valores_maximos.year, len(df)]
    for columna in columnas_especificas:
        if columna in df.columns:
            columnas_cont.append(1)
        else:
            columnas_cont.append(0)
    lista.append(columnas_cont)
    print("")

info_name = "data_preprocessed_datasets_solotrafico.csv"
lista_columnas = ['estacion', 'primer_registro','oldest_year', 'ultimo_registro', 'new_year', 'num_datos', 'CO', 'NO', 'NOX', 'NO2', 'O3', 'PM10', 'PM25', 'RH', 'SO2', 'TMP', 'WDR', 'WSP']
info_datasets = pd.DataFrame(lista, columns=lista_columnas)
info_datasets.to_csv(info_name, index=False)

AJM
2024-02-23 01:00:00 2024-05-31 19:00:00 1097 Index(['CO', 'NO', 'NOX', 'NO2', 'O3', 'PM10', 'PM25', 'RH', 'SO2', 'TMP',
       'WDR', 'WSP'],
      dtype='object')

AJU
2024-02-23 03:00:00 2024-05-31 11:00:00 1330 Index(['O3', 'PM25', 'RH', 'TMP', 'WDR', 'WSP'], dtype='object')

ATI
2024-02-23 01:00:00 2024-05-31 16:00:00 2068 Index(['CO', 'NO', 'NOX', 'NO2', 'O3', 'PM10', 'SO2'], dtype='object')

BJU
2024-02-23 01:00:00 2024-05-31 16:00:00 1449 Index(['CO', 'NO2', 'O3', 'PM10', 'PM25', 'RH', 'SO2', 'TMP', 'WDR', 'WSP'], dtype='object')

CAM
2024-02-23 02:00:00 2024-05-24 15:00:00 1208 Index(['CO', 'NO', 'NOX', 'NO2', 'O3', 'PM10', 'PM25', 'SO2'], dtype='object')

CCA
2024-02-23 02:00:00 2024-05-31 17:00:00 2083 Index(['CO', 'NO', 'NOX', 'NO2', 'O3', 'PM25', 'SO2'], dtype='object')

CHO
2024-02-28 12:00:00 2024-05-22 11:00:00 56 Index(['CO', 'NO', 'NOX', 'NO2', 'O3', 'PM10', 'SO2', 'WDR', 'WSP'], dtype='object')

CUA
2024-02-23 02:00:00 2024-03-14 09:00:00 422 Index(['CO', 'NO', 'N

# Otra sección

In [ ]:
#trainfiles_route = os.path.join(os.pardir, 'preprocessing_airpollution_data/entrenamiento_datos_por_estacion_sin_NaN')
#testfiles_route = '..\\preprocessing_airpollution_data/prueba_datos_por_estacion_sin_NaN'
trainfiles_route = os.path.join(os.pardir, 'preprocessing_airpollution_data/entrenamiento_datos_por_estacion')
testfiles_route = '..\\preprocessing_airpollution_data/prueba_datos_por_estacion'
archivos = os.listdir(trainfiles_route)
archivos = ["PED.csv"]
# Acceder a cada archivo CSV
for archivo in archivos:
    train_dirfile = os.path.join(trainfiles_route, archivo)
    station_name = archivo.replace(".csv","")
    print(station_name)
    df_train = pd.read_csv(train_dirfile)
    test_dirfile = os.path.join(testfiles_route, station_name+".csv")
    df_test = pd.read_csv(test_dirfile)
df = pd.concat([df_train, df_test], axis = 0)
print(len(df), len(df_train), len(df_test))

In [ ]:
new_nameTrain = "PED_train_normalized.csv"
new_nameTest = "PED_test_normalized.csv"
#train_procentage = round(len(df)*.8)
#test_porcentage = round(len(df)*.05)
#train_set = df[0:train_procentage]
#val_set = df[train_procentage:train_procentage+validation_porcentage]
#test_set = df[train_procentage+validation_porcentage:len(df)]
df["date"] = df["date"].str.replace("-", "/")
df['date'] = pd.to_datetime(df['date'], format='%Y/%m/%d %H:%M:%S')
fecha_limite = pd.Timestamp('2023-04-01')
train_set = df[df['date'] < fecha_limite].copy()
test_set = df[df['date'] > fecha_limite].copy()
print(len(train_set),  len(test_set))
print(len(train_set)+ len(test_set))

In [ ]:
train_set.to_csv(new_nameTrain, index=False)
test_set.to_csv(new_nameTest, index=False)

Los segmentamos en entrenamiento y prueba (se dejaron para prueba los datos de hace un año)

In [12]:
dir = "datos_por_estacion_pre_processing"
new_dirTest = "prueba_datos_por_estacion"
new_dirTrain = "entrenamiento_datos_por_estacion"

if not os.path.exists(new_dirTest):
    os.makedirs(new_dirTest)
    
if not os.path.exists(new_dirTrain):
    os.makedirs(new_dirTrain)

info_datasets = []
columnas = ["len_dataset_withoutNaN","len_trainSet_withoutNaN","len_testSet_withoutNaN"]
files = os.listdir(dir)
for file_name in files:
    file_path = os.path.join(dir,file_name)
    print(file_path)
    dataset = pd.read_csv(file_path)
    if len(dataset)>0:
        dataset[['year', 'month', 'day']] = dataset['date'].str.split('/', expand=True)
        dataset["hour"] = dataset["day"]
        dataset[["day", "hour"]] = dataset["day"].str.split(' ', expand=True)
        dataset[["hour","minute"]] = dataset["hour"].str.split(':', expand=True)
        #Ver si se quita o no
        #dataset['date'] = pd.to_datetime(dataset['date'], format='%Y/%m/%d %H:%M:%S')
        dataset['date'] = pd.to_datetime(dataset['date'])
        #dataset['date'] = pd.to_datetime(dataset[['year', 'month', 'day', 'hour','minute']])
        #dataset['date'] = pd.to_datetime(dataset['date'])
        fecha_limite = pd.Timestamp('2023-04-01')
        train_set = dataset[dataset['date'] < fecha_limite].copy()
        test_set = dataset[dataset['date'] > fecha_limite].copy()

        new_nameTrain = os.path.join(new_dirTrain, file_name)
        new_nameTest = os.path.join(new_dirTest, file_name)
        train_set.to_csv(new_nameTrain, index=False)
        test_set.to_csv(new_nameTest, index=False)
        
        info_datasets.append([len(dataset), len(train_set), len(test_set)])

    

datos_por_estacion_sin_NaN\ACO.csv
datos_por_estacion_sin_NaN\AJM.csv
datos_por_estacion_sin_NaN\AJU.csv
datos_por_estacion_sin_NaN\ARA.csv
datos_por_estacion_sin_NaN\ATI.csv
datos_por_estacion_sin_NaN\AZC.csv
datos_por_estacion_sin_NaN\BJU.csv
datos_por_estacion_sin_NaN\CAM.csv
datos_por_estacion_sin_NaN\CCA.csv
datos_por_estacion_sin_NaN\CES.csv
datos_por_estacion_sin_NaN\CHO.csv
datos_por_estacion_sin_NaN\COY.csv
datos_por_estacion_sin_NaN\CUA.csv
datos_por_estacion_sin_NaN\CUT.csv
datos_por_estacion_sin_NaN\FAC.csv
datos_por_estacion_sin_NaN\FAR.csv
datos_por_estacion_sin_NaN\GAM.csv
datos_por_estacion_sin_NaN\HGM.csv
datos_por_estacion_sin_NaN\IMP.csv
datos_por_estacion_sin_NaN\INN.csv
datos_por_estacion_sin_NaN\IZT.csv
datos_por_estacion_sin_NaN\LAG.csv
datos_por_estacion_sin_NaN\LLA.csv
datos_por_estacion_sin_NaN\LPR.csv
datos_por_estacion_sin_NaN\MER.csv
datos_por_estacion_sin_NaN\MGH.csv
datos_por_estacion_sin_NaN\MON.csv
datos_por_estacion_sin_NaN\MPA.csv
datos_por_estacion_s

In [18]:
info_name = "info_split_datasets.csv"
info_data = pd.read_csv(info_name)
info_datasets = pd.DataFrame(info_datasets, columns=columnas)
info_data = pd.concat([info_data,info_datasets], axis=1)
info_datasets.to_csv(info_name, index=False)
info_data

,file_name,len_dataset,%datosFaltantes,len_dataset_withoutNaN,len_trainSet_withoutNaN,len_testSet_withoutNaN
0,ACO.csv,193368,33.19,4588.0,4588.0,0.0
1,AJM.csv,109937,25.45,36217.0,30376.0,5840.0
2,AJU.csv,97827,43.39,5397.0,5397.0,0.0
3,ARA.csv,41899,59.76,54796.0,49306.0,5490.0
4,ATI.csv,245317,42.49,9448.0,9448.0,0.0
5,AZC.csv,81505,44.70,65234.0,59859.0,5374.0
6,BJU.csv,122678,32.91,59257.0,52096.0,7160.0
7,CAM.csv,218132,40.68,15613.0,13470.0,2143.0
8,CCA.csv,134418,36.57,40219.0,40219.0,0.0
9,CES.csv,85108,31.79,42704.0,37768.0,4936.0
